In [45]:
import matplotlib.pyplot as plt
import numpy as np
import random

## Classes & Functions

#### Algorithms

In [ ]:
class SimulatedAnnealing:
    def __init__(self, current, temperature, nodes, steps):
        self.initial_solution = current
        self.current_solution = current
        self.temperature = temperature
        self.steps = steps
        self.nodes = nodes
        self.n = len(self.initial_solution)
        self.rate = 0.99

        self.current_score = self.distance(self.initial_solution)

    def distance(self, solution) -> int:
        distance = 0
        for i in range(1, self.n):
            distance += self.nodes[solution[i] - 1].distance(self.nodes[solution[i - 1] - 1])
        return distance

    def update_temperature(self) -> None:
        self.temperature = self.temperature * self.rate

    def swap(self, solution) -> list:
        r_1, r_2, r_3, r_4 = np.random.randint(0, self.n, 4)
        solution[r_1], solution[r_2], solution[r_3], solution[r_4] = solution[r_4], solution[r_3], solution[r_2], solution[r_1]
        return solution

    def run(self) -> list:
        for _ in range(self.steps):
            cur_sol = self.current_solution
            new_solution = self.swap(cur_sol)
            new_score = self.distance(new_solution)
            print(new_score)
            print(f'the current{self.current_score}')

            if self.current_score > new_score:
                self.current_score = new_score
                self.current_solution = new_solution
            else:
                score_diff = self.current_score - new_score
                if np.exp(score_diff/self.temperature) > random.random():
                    self.current_solution = new_solution
                    self.current_score = new_score

            self.update_temperature()

        return self.current_solution, self.current_score

In [86]:
def create_random(starting_list) -> list:
    """
    Creates random initial solution, it shuffles the list and returns it.
    """

    random.shuffle(starting_list)
    return starting_list

#### Network

In [31]:
class Node:
    def __init__(self, coordinates, number):
        self.number = number
        self.x, self.y = coordinates
        # self.connections = {}

    def distance(self, b) -> int:
        return np.linalg.norm(np.array(self.get_coordinates())
            - np.array(b.get_coordinates()))

    def get_coordinates(self) -> list:
        return [self.x, self.y]

#### Helper

In [68]:
def load_data(file_name) -> dict:
    """
    Loads in a text file with the given file name and creates a node for every
    point. Returns a list of nodes.
    """
    file = open(f'TSP-Configurations/{file_name}.tsp.txt', "r")
    file = file.read().splitlines()[6:-1]
    nodes = dict()
    ids = list()

    # Go through every line of the file and create node for every line.
    for node in file:
        values = [int(value) for value in node.split()]
        nodes[values[0]] = Node([values[1], values[2]], values[0])
        ids.append(values[0])

    # Return the list of nodes.
    return nodes, ids


In [ ]:
def distance_travelled(solution, nodes) -> int:
    return sum([nodes[solution[i]].distance(nodes[solution[i - 1]])] for i in range(1, len(solution)))

In [56]:
def calculate_optimal(nodes, file_name) -> float:
    """
    Takes the dictionary of nodes and calculates the distance travelled of the
    optimal tour. This serves as a benchmark for the Simulated Annealing
    algorithm.
    """

    optimal = open(f'TSP-Configurations/{file_name}.opt.tour.txt', "r")
    optimal = optimal.read().splitlines()[5:-1]
    return sum([nodes[int(optimal[i])].distance(nodes[abs(int(optimal[i + 1]))]) for i in range(len(optimal) - 1)])

In [84]:
def calculate_current_score(nodes, current_state) -> float:
    distance = 0

    for i in range(len(current_state) - 1):
        distance += nodes[current_state[i]].distance(nodes[current_state[i + 1]])

    return distance


## Testing

Testing happens on the configuration of the eli51.tsp file. For experiments the bigger
network a280.tsp is used. 

In [62]:
file_name = 'eil51'

In [69]:
# Load in data and create the connections.
nodes, initial_list = load_data(file_name)
# nodes = create_connections(data)

In [77]:
random_start = create_random(initial_list)

In [85]:
calculate_current_score(nodes, random_start)

1560.3536229040096

In [64]:
calculate_optimal(nodes, file_name)

429.98331198338406

## Experiments